In [79]:
import pandas as pd
import numpy as np
import glob
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

import warnings  
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)  
warnings.simplefilter(action='ignore', category=FutureWarning)  

In [67]:
train1 = pd.read_csv("./train/train1.csv")
train2 = pd.read_csv("./train/train2.csv")

test1 = pd.read_csv("./test/test1.csv")
test2 = pd.read_csv("./test/test2.csv")

submit1 = pd.read_csv("./submit/submit1.csv")
submit2 = pd.read_csv("./submit/submit2.csv")

In [68]:
test1.columns

Index(['TIME', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'T1-1', 'T1-2', 'T1-3',
       'T1-4', 'T1-5', 'T1-6', 'T1-7', 'T1-8', 'T1-9', 'T1-10', 'T1-11',
       'T1-12', 'T1-13', 'T1-14', 'T1-15', 'T1-16', 'T1-17', 'T2-1', 'T2-2',
       'T2-3', 'T2-4', 'T2-5', 'T2-6', 'T2-7', 'T2-8', 'T2-9', 'T2-10',
       'T2-11', 'T2-12', 'T2-13', 'T2-14', 'T2-15', 'T2-16', 'T2-17'],
      dtype='object')

In [69]:
train1.columns

Index(['TIME', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'T3-1', 'T3-2', 'T3-3',
       'T3-4', 'T3-5', 'T3-6', 'T3-7', 'T3-8', 'T3-9', 'T3-10', 'T3-11',
       'T3-12', 'T3-13', 'T3-14', 'T3-15', 'T3-16', 'T3-17', 'T1-1', 'T1-2',
       'T1-3', 'T1-4', 'T1-5', 'T1-6', 'T1-7', 'T1-8', 'T1-9', 'T1-10',
       'T1-11', 'T1-12', 'T1-13', 'T1-14', 'T1-15', 'T1-16', 'T1-17', 'T4-1',
       'T4-2', 'T4-3', 'T4-4', 'T4-5', 'T4-6', 'T4-7', 'T4-8', 'T4-9', 'T4-10',
       'T4-11', 'T4-12', 'T4-13', 'T4-14', 'T4-15', 'T4-16', 'T4-17', 'T2-1',
       'T2-2', 'T2-3', 'T2-4', 'T2-5', 'T2-6', 'T2-7', 'T2-8', 'T2-9', 'T2-10',
       'T2-11', 'T2-12', 'T2-13', 'T2-14', 'T2-15', 'T2-16', 'T2-17'],
      dtype='object')

In [70]:
submit1.columns

Index(['TIME', 'T3-1', 'T3-2', 'T3-3', 'T3-4', 'T3-5', 'T3-6', 'T3-7', 'T3-8',
       'T3-9', 'T3-10', 'T3-11', 'T3-12', 'T3-13', 'T3-14', 'T3-15', 'T3-16',
       'T3-17', 'T4-1', 'T4-2', 'T4-3', 'T4-4', 'T4-5', 'T4-6', 'T4-7', 'T4-8',
       'T4-9', 'T4-10', 'T4-11', 'T4-12', 'T4-13', 'T4-14', 'T4-15', 'T4-16',
       'T4-17'],
      dtype='object')

In [71]:
for col in train1.columns:
    if col not in submit1.columns:
        train1[col + "_shift1"] = train1[col].shift(1).values
        train1[col + "_shift2"] = train1[col].shift(2).values
        train1[col + "_diff1"] = train1[col].diff(1).values
        train1[col + "_diff12"] = train1[col].diff(2).values

for col in test1.columns:
    if col not in submit1.columns:
        test1[col + "_shift1"] = test1[col].shift(1).values
        test1[col + "_shift2"] = test1[col].shift(2).values
        test1[col + "_diff1"] = test1[col].diff(1).values
        test1[col + "_diff12"] = test1[col].diff(2).values


for col in train2.columns:
    if col not in submit2.columns:
        train2[col + "_shift1"] = train2[col].shift(1).values
        train2[col + "_shift2"] = train2[col].shift(2).values
        train2[col + "_diff1"] = train2[col].diff(1).values
        train2[col + "_diff12"] = train2[col].diff(2).values

for col in test2.columns:
    if col not in submit2.columns:
        test2[col + "_shift1"] = test2[col].shift(1).values
        test2[col + "_shift2"] = test2[col].shift(2).values
        test2[col + "_diff1"] = test2[col].diff(1).values
        test2[col + "_diff12"] = test2[col].diff(2).values

In [72]:
model1 = XGBRegressor()
model1.fit(
    train1.drop(submit1.columns, axis=1),
    train1[submit1.columns[1:]]
)

model2 = XGBRegressor()
model2.fit(
    train2.drop(submit2.columns, axis=1),
    train2[submit2.columns[1:]]
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [83]:
submit1.iloc[:, 1:] = model1.predict(test1[train1.drop(submit1.columns, axis=1).columns])
submit2.iloc[:, 1:] = model2.predict(test2[train2.drop(submit2.columns, axis=1).columns])

In [84]:
submit1.to_csv("./submit/submit1.csv", index=None)
submit2.to_csv("./submit/submit2.csv", index=None)

In [86]:
!\rm submit.zip
!\rm -rf submit/.ipynb_checkpoints/
!zip -r submit.zip submit

  adding: submit/ (stored 0%)
  adding: submit/submit1.csv (deflated 58%)
  adding: submit/submit2.csv (deflated 63%)
